Good inspriration: https://medium.com/analytics-vidhya/breast-canc er-diagnostic-dataset-eda-fa0de80f15bd

# Implementation - Assignment 5 Diagnostic systems

Eickhoss summary of signs: 

"The size of cells tends to be homogeneous given a speciﬁc type of tissue. 

1) The presence of signiﬁcantly larger cells is evidence for the uncontrolled growth that is indicative of malignant tumors. 

2) The shape of benign cells usually shows only limited variance, whereas malignant cells can develop arbitrary structures that do not conform with the general pattern of their surroundings. 

3) The color of the cell nucleus should be identical for regular cells of the same type. Cancer cells often have signiﬁcantly larger and darker nuclei that are more densely packed with DNA. 

4) Regular cells show similar texture. Malignant tumors, on the other hand, can range from smooth surfaces to ragged or lumpy textures for neighbouring cells. 

5) Finally, for healthy tissue, cell arrangement tends to be orderly, with regular distances between cells. Cancer cells can spread out or clutter almost arbitrarily.

## Rule based classifier

In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

breast_cancer_df = pd.read_pickle("wdbc.pkl")
breast_cancer_df.head()
#print(df)

X = breast_cancer_df.drop(['malignant', 'id'], axis=1)
Y = breast_cancer_df['malignant']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=420)

In [24]:
breast_cancer_df.describe()

,id,malignant,radius_0,texture_0,perimeter_0,area_0,smoothness_0,compactness_0,concavity_0,concave points_0,...,radius_2,texture_2,perimeter_2,area_2,smoothness_2,compactness_2,concavity_2,concave points_2,symmetry_2,fractal dimension_2
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,0.372583,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,0.483918,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,0.000000,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,0.000000,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,0.000000,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,1.000000,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,1.000000,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [23]:
# no missing values &  Class distribution: 357 benign, 212 malignant
#print("\nNull Values:\n", df.isnull().sum())
#print("\nMissing Values:\n", df.isna().sum())


In [32]:
def size_control(benign_df, malignant_df, X_data):
    """ Take more features into account? 
    """
    #print(X_data)
    mean_b = benign_df['area_0'].mean() #assuming that area is bigger as absnomality forecasts
    mean_m = malignant_df['area_0'].mean()
    if (mean_m < mean_b):
        print("ERROR: Assumption doesn't hold")
    threshold =  mean_b + (mean_m-mean_b)/2
    #print(threshold, mean_b, mean_m)
    
    if (X_data['area_0'] > threshold):
        return True
    else: 
        return False

In [29]:
def rule_based_clf(X_tr, Y_tr, X_te): 
    """
    Will check for abnormalities and if any one of these activate then malignant, 
    else benign  
    """    
    # separate Bedning/Malignant 
    b_df = breast_cancer_df[breast_cancer_df.malignant == 0]
    m_df = breast_cancer_df[breast_cancer_df.malignant == 1]
    
    #predictive part
    Y_pred =[]

    for candidate in range(X_te.shape[0]): # for every test_instance/row  (171)
        #print(candidate)

        size_guess = size_control(b_df,m_df, X_te.iloc[candidate]) #give method test instance
        # do all the other controls 

        if (size_guess):
            print("Predict positive")
            Y_pred.append(1)

        else:
            #print("Predict negative")
            Y_pred.append(0) 

    return Y_pred


In [ ]:
def prediction(Y_guesses, Y_test):
    count =0
    if len(Y_guesses) != len(Y_test):
        print("Error: not compatible sizes")

    else: 
        for instance in range(Y_test.shape[0]):
            if Y_guesses
            count +=1

    accuracy = count/ len(Y_test) 
    
    reutrn count, accuracy 

In [36]:
Y_test.shape[0]

171

In [34]:
prediction = rule_based_clf(X_tr=X_train, Y_tr=Y_train, X_te=X_test.iloc[0:10])
print(prediction)

720.5833055863854 462.7901960784313 978.3764150943397
Predict negative
720.5833055863854 462.7901960784313 978.3764150943397
Predict positive
720.5833055863854 462.7901960784313 978.3764150943397
Predict positive
720.5833055863854 462.7901960784313 978.3764150943397
Predict negative
720.5833055863854 462.7901960784313 978.3764150943397
Predict positive
720.5833055863854 462.7901960784313 978.3764150943397
Predict negative
720.5833055863854 462.7901960784313 978.3764150943397
Predict negative
720.5833055863854 462.7901960784313 978.3764150943397
Predict negative
720.5833055863854 462.7901960784313 978.3764150943397
Predict negative
720.5833055863854 462.7901960784313 978.3764150943397
Predict positive
['0', '1', '1', '0', '1', '0', '0', '0', '0', '1']


In [35]:
print(Y_test.iloc[:10])

436    0
264    1
121    1
96     0
250    1
50     0
362    0
64     1
426    0
492    1
Name: malignant, dtype: int32


In [ ]:
"""
clf.fit(X_train,Y_train) # acess to all three mean, std, worst
clf.predict(X_test) 3
scores = cross_val_score(clf, X_train, Y_train) 
"""

Tips: 

Relevant features were selected using an exhaustive search in the space of 1-4 features and 1-3 separating planes.

## Random forest classifier

## Our own design

Interpretability and classification performance --> both at a certain satisfying level. (typical low interpretability: NNs, SVC (nonlinear)?, KNN?)

Here we might want to look at the initial souce again and consider the features from the first article. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3c86cef2-4437-40b3-85f1-a95beaad9a14' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>